In [1]:
!pip install nitrado "steam[client]" requests python-dotenv
%config IPCompleter.greedy=True

**1> collate workshop_id to mod_id(s) from source of truth collection**

In [2]:
# Find and collate workshop_ids: mod_ids, store 
#data in an output var
import requests
import subprocess
import re
from os import path

collection_id = '2903054110'
getCollections = 'https://api.steampowered.com/ISteamRemoteStorage/GetCollectionDetails/v1/?'
getFileDetails = 'https://api.steampowered.com/ISteamRemoteStorage/GetPublishedFileDetails/v1/?'


home_dir = path.expanduser('~')

# output contains the resulting list of tuples [(workshop_id, mod_id, mod_id, ...), ...] produced
# in this sell.
output = list()

# collection_data is a locally consumed var and contains the post-processed collection metadata response
collection_data = list()
with open(path.join(home_dir, '.steam', 'token'), 'r') as f:
    auth_header = {'Authentication': f'Bearer {f.read().strip()}'}
    resp = requests.post(
        getCollections, 
        headers=auth_header,
        data={
            'publishedfileids[0]':f'{collection_id}',
            'collectioncount': '1',
        }
    )
    collection_data = resp.json()['response']['collectiondetails']


for child in collection_data[0]['children']:
    child_id = child["publishedfileid"]
    resp = requests.post(
        getFileDetails,
        headers=auth_header,
        data={
            'itemcount': '1',
            'publishedfileids[0]': child_id,
        }
    )
    desc = resp.json()['response']['publishedfiledetails'][0]['description']
    match = re.findall(r'\nMod ID: *(\w+)', desc)
    if match is None:
        print(f'failed to find matching mod id string, mod file: {child_id}')
        quit(1)

    o = [child_id]
    for m in match:
        o.append(m)
    print(o)
    output.append(tuple(o))

['2901552077', 'mrnvsbhltr', 'mrnvsbhltrvn', 'mrnvsbhltwc']
['2820757649', '10YL_ALL', '10YL_HIGH', '10YL_LOW', '10YL_MAPS', '10YL_MEDIUM', '10YL_VH']
['2901962885', 'REORDER_CONTAINERS']
['2629286881', 'MilPoncho']
['2313387159', 'BetterSortCC']
['2282429356', 'autotsartrailers']
['2732662310', 'improvedhairmenu']
['1619603097', 'eris_minimap']
['2423906082', 'BCGTools']
['2487022075', 'TMC_TrueActions']
['499795221', 'cheatmenu', 'cheatmenu']
['2195155059', 'WestPointExpansion']
['2863949128', 'CanteensAndBottles']
['2864231031', 'SlowConsumption']
['2701170568', 'ExtraMapSymbols', 'ExtraMapSymbolsUI']
['2875848298', 'BB_CommonSense']
['2874678809', 'Lingering']
['2685168362', 'MoreDescriptionForTraits4166']
['2902192016', 'TSW_ScreamsOfPain']


**^^TODO create radial buttons per mod, per variant**

**2> Fetch the server config file**

In [3]:
from os import path
from ftplib import FTP
from configparser import ConfigParser, MissingSectionHeaderError
import io
import re

nitrado_dir = path.join(home_dir, ".nitrado")
nitrado_ftp_url = 'usmi440.gamedata.io'
ftp_config = ConfigParser()
ftp_config.read(path.join(nitrado_dir, "ftp.ini"))
section = 'dummy'


pz_cfg_src = path.join('/zomboid/profile/Zomboid/Server/servertest.ini')
pz_config = ConfigParser(allow_no_value=True)

buf = io.BytesIO()
buf.write(f'[{section}]\n'.encode())
with FTP(nitrado_ftp_url) as ftp:
    ftp.login(
        user = ftp_config['DEFAULT']['username'], 
        passwd = ftp_config['DEFAULT']['password']
    )
    ftp.retrbinary(f'RETR {pz_cfg_src}', buf.write)
    buf.seek(0)

# Remember to remove the dummy section header before writing file to ftp
pz_config.read_string(buf.read().decode())

**3> Merge the collection into the pz_config vars**

In [9]:
mod_list_sep = ','
ws_item_sep = ';'


merge_mods = pz_config[f'{section}']['mods'].split(mod_list_sep)
merge_ws_items = pz_config[f'{section}']['workshopitems'].split(ws_item_sep)

# Add new mods (i.e. mods not found in the config)
for o in output:
    ws_item = o[0]
    mods = o[1:]
    if ws_item not in merge_ws_items:
        print(f"{ws_item}({mods[0]}) not found, appending")
        merge_ws_items.append(ws_item)
        merge_mods.append(mods[0])
    else:
        continue
        print(f'{ws_item} already added, skipping')
        
# Remove old mods (i.e. mods not found in the collection)
for i in range(len(merge_ws_items) - 1):
    if i < 0:
        print('got zero len list')
        quit()
    if merge_ws_items[i] not in [o[0] for o in output]:
        print(f'workshopitem {merge_ws_items[i]}({merge_mods[i]}) not found in existing config, removing')
        del merge_ws_items[i]
        del merge_mods[i]
        
if len(merge_mods) is not len(merge_ws_items):
    print(f'something went wrong, len(merge_mods)[{len(merge_mods)}] != len(merge_ws_items)[{len(merge_ws_items)}]')
    quit(1)

#TODO doing something wrong here, not correctly assigned?
pz_config.set(f'{section}', 'Workshopitems', ws_item_sep.join(merge_ws_items))
pz_config.set(f'{section}', 'Mods', mod_list_sep.join(merge_mods))

# We're ready to go

[('pvp', 'true'), ('pauseempty', 'true'), ('globalchat', 'true'), ('chatstreams', 's,r,a,w,y,sh,f,all'), ('open', 'true'), ('serverwelcomemessage', '<RGB:1,0,0> Welcome to Project Zomboid MP ! to chat locally press "t", to global chat press "y" or add "/all" before chatting <LINE> Press /help to have a list of server commands <LINE> <RGB:1,1,1>'), ('autocreateuserinwhitelist', 'false'), ('displayusername', 'true'), ('showfirstandlastname', 'false'), ('spawnpoint', '0,0,0'), ('safetysystem', 'true'), ('showsafety', 'true'), ('safetytoggletimer', '100'), ('safetycooldowntimer', '120'), ('spawnitems', ''), ('defaultport', '21800'), ('udpport', '21801'), ('resetid', '354415'), ('mods', 'MoreDescriptionForTraits4166,mrnvsbhltr,10YL_ALL,REORDER_CONTAINERS,MilPoncho,BetterSortCC,autotsartrailers,improvedhairmenu,eris_minimap,BCGTools,TMC_TrueActions,cheatmenu,WestPointExpansion,CanteensAndBottles,SlowConsumption,ExtraMapSymbols,BB_CommonSense,Lingering,TSW_ScreamsOfPain'), ('map', 'Muldraugh,

In [12]:
print('Should we push?')
input('press [enter]')

Should we push?


press [enter] 


''

**4> Write the pz config back to the ftp**

In [50]:
import tempfile

with FTP(nitrado_ftp_url) as ftp:
    ftp.login(
        user = ftp_config['DEFAULT']['username'], 
        passwd = ftp_config['DEFAULT']['password']
    )
    with open('test.ini', 'w') as f:
        pz_config.write(f)

    with open('test.ini', 'rb') as f:
        ftp.storlines(cmd=b'STOR /foobar.txt', fp=f)

TypeError: a bytes-like object is required, not 'str'